In [3]:
!pip install pytorch-pretrained-bert
!pip install PyMuPDF

     |████████████████████████████████| 6.4 MB 21.8 MB/s 


In [4]:
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
import torch
import argparse
import numpy as np
import fitz # pymupdf(讀pdf的套件)
import re #用以同時刪除多種不同字元

In [5]:
# 讀pdf
pdf_document = "testing.pdf"
doc = fitz.open(pdf_document)
pdf_text = ""

for i in range(0,doc.pageCount): #一頁一頁讀
    page = doc.loadPage(i) #讀第i頁的文字
    pagetext = page.getText("text")
    pdf_text += re.sub('[\n]', '', pagetext) #去除換行符號

# print("pdf text: ",pdf_text)
text_list = re.split('\.',pdf_text) # 使用"."來分隔文字
#以句號為標準來判斷做句子分割(解決小數點問題)
for i in range(len(text_list)):
    if(i!=0 and i<len(text_list)):
        # print('text_list content:',text_list[i]) # self testing
        # 若分割後的句子「最後一個字元為數字，且下一句的第一個字元亦為數字」，表示這兩句之間的”.”是小數點而非句號
        if( text_list[i-1][-1].isdigit() and text_list[i][0].isdigit() ):
            # print('i=',i) # self testing
            # 將兩句合併回一個句子
            text_list[i-1] = text_list[i-1] + "." + text_list[i]
            del text_list[i]

# 用filter過濾掉text_list中句子長度小於等於6的句子
sentence_list = list(filter(lambda x: len(x) > 6, text_list))

In [6]:
# 使用GPU環境
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tag2idx = {'B': 0, 'I': 1, 'O': 2}
tags_vals = ['B', 'I', 'O']

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

def keywordextract(sentence, path):
    text = sentence
    tkns = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tkns)
    segments_ids = [0] * len(tkns)
    tokens_tensor = torch.tensor([indexed_tokens]).to(device)
    segments_tensors = torch.tensor([segments_ids]).to(device)
    model = torch.load(path)
    model.eval()
    prediction = []
    logit = model(tokens_tensor, token_type_ids=None, attention_mask=segments_tensors)
    logit = logit.detach().cpu().numpy()
    prediction.extend([list(p) for p in np.argmax(logit, axis=2)])
    for k, j in enumerate(prediction[0]):
        if j==1 or j==0:
            print(tokenizer.convert_ids_to_tokens(tokens_tensor[0].to('cpu').numpy())[k], j)

100%|██████████| 407873900/407873900 [00:08<00:00, 48181566.48B/s]


In [7]:
import sys
sys.argv=['']
del sys
# 以一個句子為單位將文章一句一句輸入
for i in range(len(sentence_list)):
    print("sentence",i+1,"keyword:")
    parser = argparse.ArgumentParser(description='BERT Keyword Extractor')

    parser.add_argument('--sentence', type=str, default= sentence_list[i] , help='sentence to get keywords')

    parser.add_argument('--path', type=str, default='/content/drive/MyDrive/model.pt',help='path to load model')

    args = parser.parse_args()

    keywordextract(args.sentence, args.path)

sentence 1 keyword:
language 0
processing 1
sentence 2 keyword:
language 0
sentence 3 keyword:
the 0
language 1
sentence 4 keyword:
language 0
model 1
sentence 5 keyword:
sentence 6 keyword:
text 0
sentence 7 keyword:
language 0
processing 1
